# ELLE - Elastic Learning

## Basics (can be ignored)

### Imports

In [ ]:
import numpy as np

### Libs

In [ ]:
class MLib:
    @staticmethod
    def linear(x):
        return x

    @staticmethod
    def dlinear(x):
        return np.ones(x.shape)

    @staticmethod
    def tanh(x):
        return np.tanh(x)

    @staticmethod
    def dtanh(x):
        tan = MLib.tanh(x)
        return (1. - tan**2.)

    @staticmethod
    def sigmoid(x):
        return 1. / (1. + np.exp(-x))

    @staticmethod
    def dsigmoid(x):
        sig = MLib.sigmoid(x)
        return sig*(1.-sig)

    @staticmethod
    def softmax(x):
        e = np.exp(x)
        return e / np.sum(e)#[:, np.newaxis]

    @staticmethod
    def dsoftmax(x):
        """
        NOTE: When computing the gradient of a combination of softmax output and crossentropy error,
        use :func:`~slowlearning.dsce_error` instead.
        Computing :func:`~slowlearning.dce_error` and :func:`~slowlearning.dsoftmax` separately is computationally very inefficient
        :param x:
        :return: Jacobean matrix. partial derivatives of all outputs :func:`~slowlearning.softmax` with respect to all inputs x
        """
        p = MLib.softmax(x)
        Ds = -np.outer(p, p)
        di = np.diag_indices(len(x))
        Ds[di] = p-p**2.
        return Ds

    @staticmethod
    def rec_error(p, y):
        return 0.5 * np.sum((p - y)**2.)

    @staticmethod
    def drec_error(p, y):
        return (p - y)

    @staticmethod
    def dlinrec_error(x, y):
        return x - y

    @staticmethod
    def ceb_error(p, y):
        eps = 1e-10
        return - np.sum(y * np.log(p + eps) + (1. - y) * np.log(1. - p + eps))

    @staticmethod
    def dceb_error(p, y):
        return - y * 1. / p + (1. - y) / (1. - p)

    @staticmethod
    def dlogceb_error(x, y):
        p = MLib.sigmoid(x)
        return - y * (1. - p) + (1. - y) * p

    @staticmethod
    def cem_error(p, y):
        eps = 1e-10
        return - np.sum(y * np.log(p + eps))

    @staticmethod
    def dcem_error(p, y):
        return - y * 1. / p

    @staticmethod
    def dsofcem_error(x, y):
        return MLib.softmax(x)-y

### Utils

In [ ]:
class T_Func_Type:
    TANH = "tanh"
    LOGISTIC = "logistic"


class T_OutFunc_Type:
    SOFTMAX = "softmax"
    LINEAR = "linear" # linear output activation function in fact corresponds to gaussian output variables
    LOGISTIC = "logistic"


class T_ErrFunc_Type:
    RECONSTRUCTION = "reconstruction"
    CROSS_ENTROPY_BINOMIAL = "cross-entropy-binomial"
    CROSS_ENTROPY_MULTINOMIAL = "cross-entropy-multinomial"


class Function(object):
    def __init__(self, f, df):
        self.f = f
        self.df = df

actFuncs = {T_Func_Type.TANH: Function(MLib.tanh, MLib.dtanh),
            T_Func_Type.LOGISTIC: Function(MLib.sigmoid, MLib.dsigmoid)}

outFuncs = {T_OutFunc_Type.SOFTMAX: Function(MLib.softmax, MLib.dsoftmax),
            T_OutFunc_Type.LINEAR: Function(MLib.linear, MLib.dlinear),
            T_OutFunc_Type.LOGISTIC: Function(MLib.sigmoid, MLib.dsigmoid)}

errFuncs = {T_ErrFunc_Type.RECONSTRUCTION: Function(MLib.rec_error, MLib.drec_error),
            T_ErrFunc_Type.CROSS_ENTROPY_BINOMIAL: Function(MLib.ceb_error, MLib.dceb_error),
            T_ErrFunc_Type.CROSS_ENTROPY_MULTINOMIAL: Function(MLib.cem_error, MLib.dcem_error)}

In [ ]:
class Utils:
    @staticmethod
    def code1ofK(labels, K):
        KcodedLabels = []
        for l in labels:
            codedK = np.zeros(shape=(K,))
            codedK[int(l)] = 1.
            KcodedLabels.append(codedK)
        return KcodedLabels


class OutputModel(object):
    def __init__(self, outFuncType, errFuncType):
        self.p = outFuncs[outFuncType].f
        self.E = errFuncs[errFuncType].f
        if outFuncType == T_OutFunc_Type.SOFTMAX and errFuncType == T_ErrFunc_Type.CROSS_ENTROPY_MULTINOMIAL:
            self.dEdx = MLib.dsofcem_error
        elif outFuncType == T_OutFunc_Type.LOGISTIC and errFuncType == T_ErrFunc_Type.CROSS_ENTROPY_BINOMIAL:
            self.dEdx = MLib.dlogceb_error
        elif outFuncType == T_OutFunc_Type.LINEAR and errFuncType == T_ErrFunc_Type.RECONSTRUCTION:
            self.dEdx = MLib.dlinrec_error
        else:
            self.dEdx = lambda x, y: errFuncs[errFuncType].df(outFuncs[outFuncType].f(x), y) * outFuncs[outFuncType].df(x)

    def cost_out_grad(self, x, y):
        out = self.p(x)
        return (self.E(out, y), out, self.dEdx(x, y))

## Theory

### Softmax and Cross-Entropy

We use the softmax output function to compute class probabilites $p_k$ for each example. In a supervised $K$-multi-class classification setting, the probability for an example $x$ to belong to class $C_k$ is given a-priori by $y_k = P(C_k|x)$. We measure the deviation of predicted values $p_k$ from the target values $y_k$ by means of the cross-entropy error $L(x,y)$. The derivatives of the cross-entropy error with respect to the inputs of the softmax are given below. For implementation purposes, it's much more efficient to pre-compute the derivative of $\frac{\partial L(softmax(x),y)}{\partial x}$ instead of computing and multiplying the gradient $\frac{\partial L(p, y)}{\partial p}$ with the Jacobean $\frac{\partial softmax(x)}{\partial{x}}$

$$p_k(\mathbf{x}) = softmax_k(\mathbf{x}) = \frac{e^{x_k}}{\sum_i e^{x_i}}$$
if $k = i$:
\begin{align}
\frac{\partial p_k}{\partial x_i} &= \frac{e^{x_k}}{\sum_i e^{x_i}} + e^{x_k} \cdot - \frac{1}{(\sum_i e^{x_i})^2} \cdot e^{x_i} \\
 &= \frac{e^{x_k}}{\sum_i e^{x_i}} - \frac{e^{x_k}}{\sum_i e^{x_i}} \cdot \frac{e^{x_i}}{\sum_i e^{x_i}} \\
 &= p_k - p_k p_i\\
 &= p_k (1 - p_i) 
\end{align}

if $k \ne i$:

\begin{align}
\frac{\partial p_k}{\partial x_i} &= e^{x_k} \cdot - \frac{1}{(\sum_i e^{x_i})^2} \cdot e^{x_i} \\
 &= -p_k p_i
\end{align}
$L(\mathbf{x}, \mathbf{y}) = - \sum_k y_k \cdot \log p_k(\mathbf{x})$
\begin{align}
\frac{\partial L}{\partial x_i} &= - \sum_k y_k \cdot \frac{1}{p_k} \cdot \frac{\partial p_k}{\partial x_i} \\
 &= - y_i (1 - p_i) + \sum_{k \ne i} y_k p_i \\
 &= - y_i + \sum_k y_k p_i \\
 &= - y_i + p_i \cdot \underbrace{\sum_k y_k}_{= 1} \\
 &= p_i - y_i
\end{align}

### Codes and Decision Boundaries

each Bit represents a linear decision boundary, partitionning the space into 2 regions. Each extra Bit again partitions each existing region into another 2 regions -> d-Bits = 2^d regions (states). Density-Estimation. In this context, figure out the relationship of: Entropy, Correlation, Frequency, Dependency of Random Variables. Buzzwords: Correlation-based learning, slow feature analysis, manifold-learning.

## Autoencoder

In [ ]:
class Autoencoder():
    def __init__(self, nvis=100, nhid=50, eta=0.1, actfunc=actFuncs[T_Func_Type.TANH]):

        self.visible_size = nvis
        self.hidden_size = nhid

        self.W = np.asarray(rng.uniform(low=-4 * np.sqrt(6. / (self.hidden_size + self.visible_size)), high=4 * np.sqrt(6. / (self.hidden_size + self.visible_size)), size=(self.hidden_size, self.visible_size)), dtype=theano.config.floatX)
        self.b1 = np.zeros(shape=(self.hidden_size,), dtype=theano.config.floatX)
        self.b2 = np.zeros(shape=(self.visible_size,), dtype=theano.config.floatX)

        self.actfunc = actfunc
        self.eta = eta

    def _encode(self, x):
        return np.dot(self.W, x) + self.b1

    def _decode(self, h):
        return np.dot(self.W.T, h) + self.b2

    def _get_rec_err(self, x, z):
        return 0.5 * np.sum((x-z)**2.)

    def _get_ce_err(self, x, z):
        eps = 1e-10
        return - np.sum((x * np.log(z + eps) + (1.-x) * np.log(1.-z + eps)))

    def init_supervised(self, nout):
        self.output_size = nout
        self.Wlabel = np.asarray(rng.uniform(low=-4 * np.sqrt(6. / (self.output_size + self.hidden_size)), high=4 * np.sqrt(6. / (self.output_size + self.hidden_size)), size=(self.output_size, self.hidden_size)), dtype=theano.config.floatX)
        self.blabel = np.zeros(shape=(self.output_size,), dtype=theano.config.floatX)
        self.OutModel = OutputModel(T_OutFunc_Type.SOFTMAX, T_ErrFunc_Type.CROSS_ENTROPY_MULTINOMIAL)

    def get_cost_grad(self, batch):

        cost = 0.
        g_W = np.zeros(self.W.shape)
        g_b1 = np.zeros(self.b1.shape)
        g_b2 = np.zeros(self.b2.shape)

        batchstel = 1. / len(batch[0])

        for x in batch:
            a = self._encode(x)
            h = self.actfunc.f(a)
            p = self._decode(h)

            cost += batchstel * MLib.rec_error(p, x)

            deltaOut = batchstel * MLib.drec_error(p, x)

            g_W += np.outer(deltaOut, h).T
            g_b2 += deltaOut

            deltaHidden = np.dot(self.W, deltaOut) * self.actfunc.df(a)
            g_W += np.outer(deltaHidden, x)
            g_b1 += deltaHidden

        cost /= len(batch)
        g_W /= len(batch)
        g_b1 /= len(batch)
        g_b2 /= len(batch)

        return cost, g_W, g_b1, g_b2

    def get_supcost_grad(self, batch, targets):

        batch_cost = 0.
        g_W = np.zeros(self.W.shape)
        g_b1 = np.zeros(self.b1.shape)
        g_Wlabel = np.zeros(self.Wlabel.shape)
        g_blabel = np.zeros(self.blabel.shape)

        for i, x in enumerate(batch):
            a = self._encode(x)
            h = self.actfunc.f(a)
            o = np.dot(self.Wlabel, h) + self.blabel
            (cost, out, grad) = self.OutModel.cost_out_grad(o, targets[i])
            batch_cost += cost

            deltaOut = grad
            g_Wlabel += np.outer(deltaOut, h)
            g_blabel += deltaOut

            deltaHidden = np.dot(self.Wlabel.T, deltaOut) * self.actfunc.df(a)
            g_W += np.outer(deltaHidden, x)
            g_b1 += deltaHidden

        batch_cost /= len(batch)
        g_W /= len(batch)
        g_b1 /= len(batch)
        g_Wlabel /= len(batch)
        g_blabel /= len(batch)

        return batch_cost, g_W, g_b1, g_Wlabel, g_blabel

    def train(self, data, epochs=2, batch_size=20, freeIndex=0):
        batch_num = len(data) / batch_size

        for epoch in xrange(epochs):
            total_cost = 0.
            self.eta *= 0.99
            for i in xrange(batch_num):
                batch = data[i*batch_size : (i+1)*batch_size]
                (cost, g_W, g_b1, g_b2) = self.get_cost_grad(batch)
                total_cost += cost
                self.W[freeIndex] -= self.eta * g_W[freeIndex]
                self.b1[freeIndex] -= self.eta * g_b1[freeIndex]
                self.b2 -= self.eta * g_b2

            print "Epoch: %d" % epoch
            print (1. / batch_num) * total_cost

    def trainSupervised(self, data, targets, epochs=10, batch_size=20):

        batch_num = len(data) / batch_size

        for epoch in xrange(epochs):
            total_cost = 0.
            self.eta = 0.99
            for batchIdx in xrange(batch_num):
                batch = data[batchIdx * batch_size : (batchIdx+1) * batch_size]
                batch_targets = targets[batchIdx * batch_size : (batchIdx+1) * batch_size]
                (cost, g_W, g_b1, g_Wlabel, g_blabel) = self.get_supcost_grad(batch, batch_targets)
                total_cost += cost
                self.W -= self.eta * g_W
                self.b1 -= self.eta * g_b1
                self.Wlabel -= self.eta * g_Wlabel
                self.blabel -= self.eta * g_blabel

            print "Epoch: %d" % epoch
            print (1. / batch_num) * total_cost